In [1]:
import pandas as pd

# processed_data = pd.read_csv('processed.csv')
cpd = pd.read_csv('./cpd_data/filtered_data.csv')

/var/folders/wh/srwjqw_j5gsbl1y7xdb9jc900000gn/T/ipykernel_3530/4205963310.py:4: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  cpd = pd.read_csv('./cpd_data/filtered_data.csv')


In [21]:
# import os
# import pandas as pd
# import numpy as np
# # import ruptures as rpt

# def load_and_combine_data(path):
#     data_list = []
#     folders = ['tdcsfog', 'defog', 'notype']

#     for folder in folders:
#         folder_path = os.path.join(path, folder)
#         # counter = 0
#         for file in os.listdir(folder_path):
#             # counter += 1
#             file_path = os.path.join(folder_path, file)
#             data = pd.read_csv(file_path)
#             data['Source'] = folder
#             data['SeriesId'] = file.split('.')[0]# Use the filename (without the extension) as the SeriesId
#             # data[['dAccV', 'dAccML', 'dAccAP']] = 0
#             # data = apply_change_point_detection(data)
#             # print(f'File number: {counter}')
#             data_list.append(data)

#     combined_data = pd.concat(data_list, ignore_index=True)
#     return combined_data

# def unify_sampling_rate(data):
#     # Interpolate defog and notype data from 100Hz to 128Hz
#     def interpolate_data(df):
#         df['Time'] = pd.to_timedelta(df['Time'], unit='ms')  # Convert 'Time' column to timedelta
#         df_resampled = df.set_index('Time').resample('7.8125ms').interpolate(method='linear').reset_index()
#         return df_resampled

#     data_defog = data[data['Source'].isin(['defog', 'notype'])].groupby('SeriesId').apply(interpolate_data).reset_index(drop=True)
#     data_tdcsfog = data[data['Source'] == 'tdcsfog']

#     unified_data = pd.concat([data_tdcsfog, data_defog], ignore_index=True)
#     return unified_data


# def unify_acceleration_units(data):
#     g_to_ms2 = 9.81

#     data.loc[data['Source'].isin(['defog', 'notype']), ['AccV', 'AccML', 'AccAP']] *= g_to_ms2
#     return data

# def filter_data(data):
#     filtered_data = data[(data['Valid'].fillna(True)) & (data['Task'].fillna(True))].reset_index(drop=True)
#     return filtered_data

# data_path = "./train/"
# data = load_and_combine_data(data_path)
# # data = unify_sampling_rate(data)
# data = unify_acceleration_units(data)
# filtered_data = filter_data(data)
# # filtered_data.to_csv('processed2.csv', index=False)

In [22]:
# cpd = cpd.merge(filtered_data[['SeriesId', 'Time', 'Valid', 'Task']], on=['SeriesId', 'Time'], how='left')

In [2]:
cpd.head()

,AccV,AccML,AccAP,StartHesitation,Turn,Walking,SeriesId,Time,dAccV,dAccML,dAccAP
0,-9.533939,0.566322,-1.413525,0.0,0.0,0.0,003f117e14,0,0,0,0
1,-9.536140,0.564137,-1.440621,0.0,0.0,0.0,003f117e14,1,0,0,0
2,-9.529345,0.561765,-1.429332,0.0,0.0,0.0,003f117e14,2,0,0,0
3,-9.531239,0.564227,-1.415490,0.0,0.0,0.0,003f117e14,3,0,0,0
4,-9.540825,0.561854,-1.429471,0.0,0.0,0.0,003f117e14,4,0,0,0


In [28]:
type(str(cpd['Time'][0]))

str

In [3]:
cpd = cpd.dropna(subset=['AccV', 'AccML', 'AccAP'], how='all')
cpd.reset_index(drop=True, inplace=True)
cpd[['AccV', 'AccML', 'AccAP']] = cpd[['AccV', 'AccML', 'AccAP']].interpolate()
cpd[['StartHesitation', 'Turn', 'Walking']] = cpd[['StartHesitation', 'Turn', 'Walking']].fillna(method='ffill')
cpd['SeriesId_time'] = cpd['SeriesId'].astype(str) + '_' + cpd['Time'].astype(str)

In [4]:
# read in all the metadata
daily_metadata = pd.read_csv('daily_metadata.csv')
defog_metadata = pd.read_csv('defog_metadata.csv')
tdcsfog_metadata = pd.read_csv('tdcsfog_metadata.csv')
subjects = pd.read_csv('subjects.csv')
tasks = pd.read_csv('tasks.csv')
events = pd.read_csv('events.csv')
# print all the columns
print(daily_metadata.columns)
print(defog_metadata.columns)
print(tdcsfog_metadata.columns)
print(subjects.columns)
print(tasks.columns)
print(cpd.columns)
print(events.columns)

Index(['Id', 'Subject', 'Visit', 'Beginning of recording [00:00-23:59]'], dtype='object')
Index(['Id', 'Subject', 'Visit', 'Medication'], dtype='object')
Index(['Id', 'Subject', 'Visit', 'Test', 'Medication'], dtype='object')
Index(['Subject', 'Visit', 'Age', 'Sex', 'YearsSinceDx', 'UPDRSIII_On',
       'UPDRSIII_Off', 'NFOGQ'],
      dtype='object')
Index(['Id', 'Begin', 'End', 'Task'], dtype='object')
Index(['AccV', 'AccML', 'AccAP', 'StartHesitation', 'Turn', 'Walking',
       'SeriesId', 'Time', 'dAccV', 'dAccML', 'dAccAP', 'SeriesId_time'],
      dtype='object')
Index(['Id', 'Init', 'Completion', 'Type', 'Kinetic'], dtype='object')


In [19]:
daily_metadata['newId'] = daily_metadata['Subject'] + daily_metadata['Visit'].astype(str)
defog_metadata['newId'] = defog_metadata['Subject'] + '_' + defog_metadata['Visit'].astype(str) + '_' + defog_metadata['Medication']
tdcsfog_metadata['newId'] = (tdcsfog_metadata['Subject'] + '_' + 
                          tdcsfog_metadata['Visit'].astype(str) + '_' + 
                          tdcsfog_metadata['Test'].astype(str) + '_' + 
                          tdcsfog_metadata['Medication'])
tasks['UniqueTaskId'] = (tasks['Id'] + '_' + 
                         tasks['Begin'].astype(str) + '_' + 
                         tasks['End'].astype(str) + '_' + 
                         tasks['Task'])
subjects['newId'] = subjects['Subject'] + '_' + subjects['Visit'].astype(str)
events['newId'] = events['Subject'] + '_' + events['Visit'].astype(str) + '_' + events['Task']

TypeError: unsupported operand type(s) for +: 'Categorical' and 'str'

In [6]:
# find duplicates in the cpd seriesId_time column
# cpd = cpd.dropna(subset=['AccV', 'AccML', 'AccAP'], how='all')
cpd.duplicated(subset=['SeriesId_time'], keep=False).value_counts()

False    14547506
dtype: int64

In [10]:
# count nan values in each column
cpd.isna().sum()

AccV               0
AccML              0
AccAP              0
StartHesitation    0
Turn               0
Walking            0
SeriesId           0
Time               0
dAccV              0
dAccML             0
dAccAP             0
SeriesId_time      0
dtype: int64

In [ ]:


relationships = [
    # ("subjects", "Subject", "daily_metadata", "Subject"),
    # ("subjects", "Subject", "defog_metadata", "Subject"),
    # ("subjects", "Subject", "tdcsfog_metadata", "Subject"),
    ("tdcsfog_metadata", "Subject", "Subjects", "Subject"),
    ("daily_metadata", "Subject", "Subjects", "Subject"),
    ("defog_metadata", "Subject", "Subjects", "Subject"),
    ("daily_metadata", "Id", "tasks", "Id"),
    ("defog_metadata", "Id", "tasks", "Id"),
    ("daily_metadata", "Id", "events", "Id"),
    ("tdcsfog_metadata", "Id", "events", "Id"),
    ("tdcsfog_metadata", "Id", "tasks", "Id"),
    # ("tasks", "Id", "cpd", "SeriesId"),
    ("cpd", "SeriesId", "tasks", "Id"),
    # ("events", "Id", "cpd", "SeriesId"),
    # ("defog_metadata", "Id", "cpd", "SeriesId"),
    # ("tdcsfog_metadata", "Id", "cpd", "SeriesId")
    ("cpd", "SeriesId", "defog_metadata", "Id"),
    ("cpd", "SeriesId", "tdcsfog_metadata", "Id"),
    ("cpd", "SeriesId", "events", "Id"),
    ("cpd", "Time", "events", "Init"),
    ("cpd", "Time", "events", "Completion"),
]


In [17]:
import featuretools as ft

# Separate target labels from features
cpd_features = cpd.drop(columns=['StartHesitation', 'Turn', 'Walking'])
cpd_target = cpd[['StartHesitation', 'Turn', 'Walking', 'SeriesId_time']]

# Add the cpd_features dataframe as an entity
# Create new entityset
es = ft.EntitySet(id="parkinsons_data")

# Create entities
es = es.add_dataframe(dataframe_name="subjects",
                              dataframe=subjects,
                              index="newId")

es = es.add_dataframe(dataframe_name="daily_metadata",
                              dataframe=daily_metadata,
                              index="Id")

es = es.add_dataframe(dataframe_name="defog_metadata",
                              dataframe=defog_metadata,
                              index="Id")

es = es.add_dataframe(dataframe_name="tdcsfog_metadata",
                              dataframe=tdcsfog_metadata,
                              index="Id")

es = es.add_dataframe(dataframe_name="tasks",
                              dataframe=tasks,
                              index="UniqueTaskId",
                              time_index="Begin")

es = es.add_dataframe(dataframe_name="cpd_features",
                              dataframe=cpd_features,
                              index="SeriesId_time",
                              time_index="Time")

es = es.add_dataframe(dataframe_name="cpd_target",
                              dataframe=cpd_target,
                              index="SeriesId_time")

es = es.add_dataframe(dataframe_name="events",
                                dataframe=events,
                                index="newId")



/Users/johnny/opt/anaconda3/envs/kaggle/lib/python3.10/site-packages/featuretools/entityset/entityset.py:754: UserWarning: A Woodwork-initialized DataFrame was provided, so the following parameters were ignored: index
  warnings.warn(
/Users/johnny/opt/anaconda3/envs/kaggle/lib/python3.10/site-packages/featuretools/entityset/entityset.py:754: UserWarning: A Woodwork-initialized DataFrame was provided, so the following parameters were ignored: index, time_index
  warnings.warn(


,StartHesitation,Turn,Walking,StartHesitation + Turn,StartHesitation + Walking,Turn + Walking,StartHesitation * Turn,StartHesitation * Walking,Turn * Walking
SeriesId_time,,,,,,,,,
003f117e14_0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
003f117e14_1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
003f117e14_2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
003f117e14_3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
003f117e14_4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
e8e530a4f9_180615,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
e8e530a4f9_180616,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
e8e530a4f9_180617,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [18]:
# Run deep feature synthesis
feature_matrix, feature_defs = ft.dfs(entityset=es,
                                        relationships=relationships,
                                      target_dataframe_name='cpd_features',
                                      trans_primitives=['add_numeric', 'multiply_numeric'],
                                      agg_primitives=['mean', 'sum', 'mode'],
                                      max_depth=3)

feature_matrix

/Users/johnny/opt/anaconda3/envs/kaggle/lib/python3.10/site-packages/featuretools/synthesis/dfs.py:321: UnusedPrimitiveWarning: Some specified primitives were not used during DFS:
  agg_primitives: ['mean', 'mode', 'sum']
This may be caused by a using a value of max_depth that is too small, not setting interesting values, or it may indicate no compatible columns for the primitive were found in the data. If the DFS call contained multiple instances of a primitive in the list above, none of them were used.
  warnings.warn(warning_msg, UnusedPrimitiveWarning)


,AccV,AccML,AccAP,SeriesId,Time,dAccV,dAccML,dAccAP,AccAP + AccML,AccAP + AccV,...,AccV * Time,AccV * dAccAP,AccV * dAccML,AccV * dAccV,dAccAP * Time,dAccAP * dAccML,dAccAP * dAccV,dAccML * Time,dAccML * dAccV,dAccV * Time
SeriesId_time,,,,,,,,,,,,,,,,,,,,,
003f117e14_0,-9.533939,0.566322,-1.413525,003f117e14,0,0,0,0,-0.847204,-10.947465,...,-0.000000e+00,-0.0,-0.0,-0.0,0.0,0.0,0.0,0.0,0.0,0.0
009ee11563_0,-9.417296,0.767819,-1.758242,009ee11563,0,0,0,0,-0.990423,-11.175538,...,-0.000000e+00,-0.0,-0.0,-0.0,0.0,0.0,0.0,0.0,0.0,0.0
011322847a_0,-8.934903,-0.045351,-3.295116,011322847a,0,0,0,0,-3.340467,-12.230019,...,-0.000000e+00,-0.0,-0.0,-0.0,0.0,0.0,0.0,0.0,0.0,0.0
01d0fe7266_0,-8.741742,0.447673,-3.782152,01d0fe7266,0,0,0,0,-3.334479,-12.523894,...,-0.000000e+00,-0.0,-0.0,-0.0,0.0,0.0,0.0,0.0,0.0,0.0
024418ba39_0,-9.298495,-0.981302,-3.476819,024418ba39,0,0,0,0,-4.458120,-12.775313,...,-0.000000e+00,-0.0,-0.0,-0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
434732a5e3_442491,-8.714267,-1.518891,-4.371211,434732a5e3,442491,0,0,0,-5.890102,-13.085478,...,-3.855985e+06,-0.0,-0.0,-0.0,0.0,0.0,0.0,0.0,0.0,0.0
434732a5e3_442492,-8.736043,-1.565598,-4.518391,434732a5e3,442492,0,0,0,-6.083990,-13.254434,...,-3.865629e+06,-0.0,-0.0,-0.0,0.0,0.0,0.0,0.0,0.0,0.0
434732a5e3_442493,-8.782509,-1.598152,-4.661392,434732a5e3,442493,0,0,0,-6.259544,-13.443901,...,-3.886199e+06,-0.0,-0.0,-0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [21]:
# merge feature matrix with cpd_target
combined_matrix = feature_matrix.merge(cpd_target, on='SeriesId_time')

In [22]:
combined_matrix.corr()

,AccV,AccML,AccAP,Time,dAccV,dAccML,dAccAP,AccAP + AccML,AccAP + AccV,AccAP + Time,...,AccV * dAccV,dAccAP * Time,dAccAP * dAccML,dAccAP * dAccV,dAccML * Time,dAccML * dAccV,dAccV * Time,StartHesitation,Turn,Walking
AccV,1.000000,-0.031818,-0.028679,0.082948,-0.002694,-0.019330,-0.034181,-0.039997,0.299826,0.082948,...,0.024297,0.035461,-0.032824,-0.000856,-0.008950,-0.017737,0.005210,0.016502,0.036560,0.010456
AccML,-0.031818,1.000000,-0.071743,-0.089284,-0.044904,-0.046265,-0.032136,0.339343,-0.078881,-0.089288,...,0.041381,-0.055464,-0.013095,-0.049704,-0.061704,-0.017257,-0.039254,-0.052290,0.006402,-0.039202
AccAP,-0.028679,-0.071743,1.000000,-0.506844,-0.025412,0.007165,0.174996,0.913893,0.945003,-0.506814,...,0.027218,-0.041937,0.004294,-0.007940,-0.054561,-0.021262,-0.069359,0.158323,0.185712,0.079674
Time,0.082948,-0.089284,-0.506844,1.000000,-0.077830,-0.098471,-0.250199,-0.514339,-0.456585,1.000000,...,0.077911,-0.014999,-0.092892,-0.073579,-0.032034,-0.059554,-0.003061,-0.091458,-0.097341,0.046669
dAccV,-0.002694,-0.044904,-0.025412,-0.077830,1.000000,0.372804,0.338766,-0.042241,-0.025134,-0.077833,...,-0.994714,0.338618,0.352118,0.866512,0.304788,0.623723,0.768403,-0.018095,-0.064854,0.021076
dAccML,-0.019330,-0.046265,0.007165,-0.098471,0.372804,1.000000,0.287095,-0.012072,0.000514,-0.098473,...,-0.375973,0.188248,0.820131,0.329224,0.772625,0.633501,0.234315,-0.017370,0.030099,-0.028476
dAccAP,-0.034181,-0.032136,0.174996,-0.250199,0.338766,0.287095,1.000000,0.151958,0.155830,-0.250197,...,-0.336607,0.578819,0.391872,0.420524,0.168776,0.216098,0.237730,-0.018988,0.081925,0.022052
AccAP + AccML,-0.039997,0.339343,0.913893,-0.514339,-0.042241,-0.012072,0.151958,1.000000,0.859120,-0.514313,...,0.042512,-0.062124,-0.001280,-0.027718,-0.076569,-0.027076,-0.081388,0.128032,0.177750,0.059184
AccAP + AccV,0.299826,-0.078881,0.945003,-0.456585,-0.025134,0.000514,0.155830,0.859120,1.000000,-0.456556,...,0.033927,-0.028421,-0.006642,-0.007858,-0.055000,-0.026096,-0.064491,0.156501,0.189204,0.079461
AccAP + Time,0.082948,-0.089288,-0.506814,1.000000,-0.077833,-0.098473,-0.250197,-0.514313,-0.456556,1.000000,...,0.077914,-0.015001,-0.092893,-0.073581,-0.032037,-0.059556,-0.003064,-0.091454,-0.097336,0.046674


The change points are poorly correlated, the multiplications and additions of the acc's are better than themselves alone.
* accap + daccap
* accML + daccML
* accV x daccV